### Read training, dev and unlabeled test data

The following provides a starting code (Python 3) of how to read the labeled training and dev cipher text, and unlabeled test cipher text, into lists.

In [24]:
train, dev, test = [], [], []

In [25]:
for x in open('./train_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    # x[0] will be the label (0 or 1), and x[1] will be the ciphertext sentence.
    x[0] = int(x[0]) 
    train.append(x)
print(len(train))
print(train[:3])

16220
[[0, 'lkêcê yoúc cêêö y#êjl lw mówám Újám j Úêê# ütlk Úol lkêú z#ê ctöé8ú ówl xoóóú éê#xw#öê#c .'], [0, '6êcétlê jolêot8 zc éê#xw#öjóáê , tl zc j #jlkê# 8tcl8êcc jöÚ8ê 6wüó lkê öt668ê wx lkê #wj6 , ükê#ê lkê lkêöjltá t#wótêc j#ê lww wÚ2twoc jó6 lkê cê+oj8 éw8tltác lww cöoy .'], [0, 'tx lktc kw8t6jú öw2tê tc coééwcê6 lw Úê j ytxl , cwöêÚw6ú oóü#jééê6 tl êj#8ú , lwwm wol j88 lkê yww6 cloxx , jó6 8êxl Úêktó6 lkê á#jé ( 8tlê#j88ú ) .']]


In [26]:
for x in open('./dev_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    # x[0] will be the label (0 or 1), and x[1] will be the ciphertext sentence.
    x[0] = int(x[0]) 
    dev.append(x)
print (len(dev))
print (dev[:3])

2027
[[1, 'ów8jó Ú#j2ê8ú l#êj6c ükê#ê xêü jöê#tájó xt8öc 6j#ê lw 6ê82ê 77 tólw lkê üw#86 wx jöÚt2j8êóáê jó6 jöÚtyotlú <<<'], [0, 'ê2êó öo#ékú zc ê+éê#l áwötá ltötóy jó6 xjöê6 ákj#tcöj áj ózl #êcáoê lktc êxxw#l .'], [1, 'üt88 jcco#ê68ú #jóm jc wóê wx lkê á8ê2ê#êcl , öwcl 6êáêélt2ê8ú jöoctóy áwöê6têc wx lkê úêj# .']]


#### Different from 'train' and 'dev' that are both list of tuples, 'test' will be just a list.

In [27]:
for x in open('./test_enc_unlabeled.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r')
    test.append(x)
print (len(test))
print (test[:3])

2028
['j 6t6jáltá jó6 6o88 6wáoöêólj#ú y8w#txútóy cwxlüj#ê jój#ákú .', 'ówlktóy cltámc , #êj88ú , ê+áêél j 8tóyê#tóy á#êêétóêcc wóê xêê8c x#wö Úêtóy 6#jyyê6 lk#woyk j cj6 , cw#6t6 oót2ê#cê wx yoóc , 6#oyc , j2j#táê jó6 6jöjyê6 6#êjöc .', 'öo#ékú jó6 üt8cwó jáloj88ú öjmê j é#êllú yww6 lêjö <<< Úol lkê é#wvêál co##woó6tóy lkêö tc 6tcl#êcctóy8ú #wlê .']


#### You can split every sentence into lists of words by white spaces.

In [28]:
train_split = [[x[0], x[1].split(' ')] for x in train]
dev_split = [[x[0], x[1].split(' ')] for x in dev]
test_split = [[x.split(' ')] for x in test]

### Main Code Body

You may choose to experiment with different methods using your program. However, you need to embed the training and inference processes at here. We will use your prediction on the unlabeled test data to grade, while checking this part to understand how your method has produced the predictions.

### Imports

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from nltk.probability import FreqDist
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM, Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import model_from_json
from sklearn.model_selection import train_test_split
# from keras_self_attention import SeqSelfAttention
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Taking a Look at Data

In [2]:
train_df = pd.read_csv('train_enc.tsv', sep='\t', header = None, names = ['label', 'text'], encoding='utf-8')
test_df = pd.read_csv('dev_enc.tsv', sep='\t', header = None, names = ['label', 'text'], encoding='utf-8')

In [3]:
train_df.head()

,label,text
0,0,lkêcê yoúc cêêö y#êjl lw mówám Újám j Úêê# ütl...
1,0,"6êcétlê jolêot8 zc éê#xw#öjóáê , tl zc j #jlkê..."
2,0,tx lktc kw8t6jú öw2tê tc coééwcê6 lw Úê j ytxl...
3,1,vocl ükêó úwo lktóm lkjl ê2ê#ú éwcctÚ8ê jóy8ê ...
4,1,yt2ê á#ê6tl lw ê2ê#úwóê x#wö #wÚtócwó 6wüó lw ...


In [4]:
test_df.head()

,label,text
0,1,ów8jó Ú#j2ê8ú l#êj6c ükê#ê xêü jöê#tájó xt8öc ...
1,0,ê2êó öo#ékú zc ê+éê#l áwötá ltötóy jó6 xjöê6 á...
2,1,"üt88 jcco#ê68ú #jóm jc wóê wx lkê á8ê2ê#êcl , ..."
3,1,"tl kjc j ájxxêtójlê6 , c8wééú Ú#t88tjóáê , céj..."
4,1,lww öoák wx clw#úlê88tóy öw2êc jüjú x#wö cw8wó...


## Utility Functions

In [5]:
def tokenize_cipher(stringerino):
  stringerino = stringerino.rstrip('\n\r').split()
  tokens = [token for token in stringerino if token not in [',','.']]
  return tokens
tokenize_cipher('John Doe Was Here Lol , ! .')

['John', 'Doe', 'Was', 'Here', 'Lol', '!']

In [6]:
def preprocess_train_or_test(df, top_words, maxlen, fdist = None, train = True):
  # top_words = 20000
  # maxlen = 200 
  words_list = []
  for entry in df['text']:
    splitted = entry.split()
    for word in splitted:
      if word not in ['.', ',']:
        words_list.append(word)
  
  if not fdist:
    fdist = FreqDist(words_list)

  
  train_list = []
  train_arr = df.text.to_numpy()

  for text in train_arr:
    text = text.split()
    text = [word for word in text]
    text = " ".join(text)
    train_list.append(text)

  train_arr = np.asarray(train_list)
  df['text'] = train_arr

  #tokenizing
  df['text'] = df.text.apply(tokenize_cipher)

  # converting text tokens to freq dist ranks


  terms = [term for term, count in fdist.most_common(n=top_words)]
  df.text = df.text.apply(lambda text:
                                    [terms.index(term) if term in terms else 0 
                                    for term in text])

  x = df.text
  y = df.label
  
  x = sequence.pad_sequences(x, maxlen=maxlen)

  if train:
    return x, y, fdist
  else:
    return x, y

In [7]:
def preprocess_predict_data(df, top_words, maxlen, fdist = None):
  # top_words = 20000
  # maxlen = 200 
  words_list = []
  for entry in df['text']:
    splitted = entry.split()
    for word in splitted:
      words_list.append(word)
  
  if not fdist:
    fdist = FreqDist(words_list)

  train_list = []
  train_arr = df.text.to_numpy()

  for text in train_arr:
    text = text.split()
    text = [word for word in text]
    text = " ".join(text)
    train_list.append(text)

  train_arr = np.asarray(train_list)
  df['text'] = train_arr

  #tokenizing
  df['text'] = df.text.apply(tokenize_cipher)

  # converting text tokens to freq dist ranks


  terms = [term for term, count in fdist.most_common(n=top_words)]
  df.text = df.text.apply(lambda text:
                                    [terms.index(term) if term in terms else 0 
                                    for term in text])

  x = df.text

  x = sequence.pad_sequences(x, maxlen=maxlen)
  return x

## Defining Transformer Block (Stack of Layers)

In [17]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.feed_forward = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.normalize1 = layers.LayerNormalization(epsilon=1e-6)
        self.normalize2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):

        attention_output = self.attention(inputs, inputs)
        attention_output = self.dropout1(attention_output, training=training)
        out1 = self.normalize1(inputs + attention_output)
        feed_forward_output = self.feed_forward(out1)
        feed_forward_output = self.dropout2(feed_forward_output, training=training)
        out2 = self.normalize2(out1 + feed_forward_output)

        return out2


## Defining Embedding Layers

In [18]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

## Model Architecture

In [19]:
def get_model():
  maxlen = 200
  vocab_size = 20000
  embed_dim = 64  # Embedding size for each token
  num_heads = 2  # Number of attention heads
  ff_dim = 64  # Hidden layer size in feed forward network inside transformer

  inputs = layers.Input(shape=(maxlen,))
  embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
  x = embedding_layer(inputs)
  
  transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
  x = transformer_block(x)
  x = layers.GlobalAveragePooling1D()(x)
  x = layers.Dropout(0.3)(x)
  
  x = layers.Dense(128, activation="relu")(x)
  x = layers.Dropout(0.3)(x)
  
  # x = layers.Dense(64, activation="relu")(x)
  # x = layers.Dropout(0.5)(x)

  outputs = layers.Dense(2, activation="softmax")(x)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
  return model

### Main Block

In [ ]:
# Preprocessing
train_df = pd.read_csv('train_enc.tsv', sep='\t', header = None, names = ['label', 'text'], encoding='utf-8')
test_df = pd.read_csv('dev_enc.tsv', sep='\t', header = None, names = ['label', 'text'], encoding='utf-8')

x_train, y_train, fdist = preprocess_train_or_test(train_df, top_words=20000, maxlen=200)
x_val, y_val = preprocess_train_or_test(test_df, top_words=20000, maxlen=200, train = False, fdist = fdist)

# Define a Model Checkpoint for saving best model (based on max val_accuracy)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='./model_checkpoints/transformer_model_4',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# Initialize and Fit Model
model = get_model()
print(model.summary())
history = model.fit(
    x_train, y_train, batch_size=64, epochs=20, validation_data=(x_val, y_val), callbacks = [model_checkpoint_callback]
)

# Model Evaluation
scores = model.evaluate(x_val, y_val, verbose=0)
print("Accuracy:{}".format(scores[1]*100))

# Save the trained model
model.save('transformer_model_4')

# Load the saved model
loaded_model = tf.keras.models.load_model('transformer_model_4')

# Load the best weights from the 'best' checkpoint
loaded_model.load_weights('./model_checkpoints/transformer_model_4')

# Evaluate again with best model ver.
loaded_model.evaluate(x_val, y_val)


### Prediction on Unlabeled set

In [40]:
prediction_df = pd.read_csv('test_enc_unlabeled.tsv', sep='\t', header = None, names = ['text'], encoding='utf-8')
x_pred = preprocess_predict_data(prediction_df, top_words = 20000, maxlen = 200, fdist = fdist)
y_pred = loaded_model.predict(x_pred)
results = np.argmax(y_pred, axis = 1)
# Eventually, results need to be a list of 2028 0 or 1's
results = list(results)

### Output Prediction Result File

You will need to submit a prediction result file. It should have 2028 lines, every line should be either 0 or 1, which is your model's prediction on the respective test set instance.

In [41]:
# suppose you had your model's predictions on the 2028 test cases read from test_enc_unlabeled.tsv, and 
#those results are in the list called 'results'
assert (len(results) == 2028)

In [42]:
# make sure the results are not float numbers, but intergers 0 and 1
results = [int(x) for x in results]

In [43]:
# write your prediction results to 'upload_predictions.txt' and upload that later
with open('upload_predictions.txt', 'w', encoding = 'utf-8') as fp:
    for x in results:
        fp.write(str(x) + '\n')

In [56]:
result = np.loadtxt('upload_predictions.txt')

In [57]:
np.unique(result, return_counts = True)

(array([0., 1.]), array([1006, 1022], dtype=int64))